<a href="https://colab.research.google.com/github/cjdesidera/Bootcamp-Geracao-Tech-Unimed-BH/blob/main/Analise_COVID_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Bibliotecas**

In [ ]:
import re
import pandas as pd
import numpy as np


In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from datetime import datetime

### **DataFrame**

In [ ]:
url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

NameError: ignored

In [ ]:
df.dtypes

NameError: ignored

Nomes de colunas não devem ter letras maiúsculas e caractéres especiais. 
Formatação das colunas

In [ ]:
def corrige_colunas(col_name):
  return re.sub(r"[/ | ]","",col_name).lower()

corrige_colunas("AdgE/P ou")


NameError: ignored

Corrigindo as colunas 

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
df

,sno,observationdate,provincestate,countryregion,lastupdate,confirmed,deaths,recovered
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
26708,26709,2020-05-19,Wyoming,US,2020-05-20 02:32:19,776.0,10.0,0.0
26709,26710,2020-05-19,Xinjiang,Mainland China,2020-05-20 02:32:19,76.0,3.0,73.0
26710,26711,2020-05-19,Yukon,Canada,2020-05-20 02:32:19,11.0,0.0,11.0
26711,26712,2020-05-19,Yunnan,Mainland China,2020-05-20 02:32:19,185.0,2.0,183.0


Trabalhando com os dados do Brasil

In [ ]:
df.loc[df.countryregion == 'Brazil']

NameError: ignored

In [ ]:
brasil = df.loc[(df.countryregion =='Brazil ') & (df.confirmed > 0)]


NameError: ignored

## **Número de casos confirmados**

In [ ]:
#Gráfico da evolução de casos confirmados
px.line(brasil, 'observationdate', 'confirmed', title =' Casos Confirmados no Brasil')

In [ ]:
# Usar técnica de programção funcional
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

NameError: ignored

In [ ]:
px.lin(brasil, x='observationdate', y='novoscasos', title='Novos Casos por dia')

NameError: ignored

O número de novos casos parece ser um excelente caso para modelagem. Será que conseguimos predizer quando o número de novos casos vai começar a cair?

## **Mortes**

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes', mode='lines+markers',
              line=dict(color='red'))

SyntaxError: ignored

In [ ]:
#Edita o layout
fig.update_layout(title='Mortes por COVID-19 no Brasil',
                   xaxis_title='Data',
                   yaxis_title='Número de mortes')
fig.show()

NameError: ignored

# Calculando Taxa de Crescimento (TC)
TC=(presente/passado)**(1/n)-1


In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    # Se data_inicio for None, define como a primeira data disponível no dataset
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
    
    # Define os valores de presente e passado
    passado = data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]
    
    # Define o número de pontos no tempo q vamos avaliar
    n = (data_fim - data_inicio).days
    
    # Calcula a taxa
    taxa = (presente/passado)**(1/n) - 1

    return taxa*100

In [ ]:
#Txa de crescimento médio do COVID
taxa_crescimento(brasil, 'confirmed')

NameError: ignored

In [ ]:
cresc_medio = taxa_crescimento(brasil, 'confirmed')
print(f"O crescimento médio do COVID no Brasil no período avaliado foi de {cresc_medio.round(2)}%.")

NameError: ignored

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    data_fim = data.observationdate.max()
    n = (data_fim - data_inicio).days
    taxas = list(map(
        lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1,n+1)
    ))
    return np.array(taxas)*100

In [ ]:

tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')

NameError: ignored

In [ ]:
tx_dia

NameError: ignored

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()
px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil',
       labels={'y':'Taxa de crescimento', 'x':'Data'})

NameError: ignored

In [ ]:
px.line(x=pd.date_range(primeiro_dia, brasil.observation.max())(1:), y=tx_dia. title = 'Taxa de Crtescimento de Casos Confirmados no Brasil')

SyntaxError: ignored

### **Predições de Séries Temporais**
Vamos construir um modelo de séries temporais para prever os novos casos. Antes analisaremos a série temporal.